In [3]:
# 创建节点和边

import dgl
import torch as th

# 边 0->1, 0->2, 0->3, 1->3
u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
g = dgl.graph((u, v))
print(g) # 图中节点的数量是DGL通过给定的图的边列表中最大的点ID推断所得出的

# 获取节点的ID
print(g.nodes())

# 获取边的对应端点
print(g.edges())

# 获取边的对应端点和边ID
print(g.edges(form='all'))

# 如果具有最大ID的节点没有边，在创建图的时候，用户需要明确地指明节点的数量。
g = dgl.graph((u, v), num_nodes=8)
# 获取边的对应端点和边ID
print(g.edges(form='all'))
# 获取节点的ID
print(g.nodes())

bg = dgl.to_bidirected(g)
bg.edges()

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})
tensor([0, 1, 2, 3])
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))
tensor([0, 1, 2, 3, 4, 5, 6, 7])


(tensor([0, 0, 0, 1, 1, 2, 3, 3]), tensor([1, 2, 3, 0, 3, 0, 0, 1]))

In [4]:
# 类型

edges = th.tensor([2, 5, 3]), th.tensor([3, 5, 0])  # 边：2->3, 5->5, 3->0
g64 = dgl.graph(edges)  # DGL默认使用int64
print(g64.idtype)  

g32 = dgl.graph(edges, idtype=th.int32)  # 使用int32构建图
g32.idtype

g64_2 = g32.long()  # 转换成int64
g64_2.idtype

g32_2 = g64.int()  # 转换成int32
g32_2.idtype

torch.int64


torch.int32

In [5]:
# 属性

g = dgl.graph(([0, 0, 1, 5], [1, 2, 2, 0])) # 6个节点，4条边

g.ndata['x'] = th.ones(g.num_nodes(), 3)               # 长度为3的节点特征
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # 标量整型特征

# 不同名称的特征可以具有不同形状
g.ndata['y'] = th.randn(g.num_nodes(), 5)
g.ndata['x'][0]                  # 获取节点1的特征

g.edata['x'][th.tensor([0, 3])]  # 获取边0和3的特征

tensor([1, 1], dtype=torch.int32)

In [10]:
g.ndata['y']

tensor([[ 0.0908,  0.0872,  0.2179,  1.4998,  2.2404],
        [-0.8600,  1.6146, -0.3041, -0.7136, -0.2585],
        [-0.7883,  0.0594, -0.8250,  0.5374, -1.5971],
        [-0.5913, -0.1998, -0.8382,  0.8527, -0.5278],
        [ 0.0944,  0.5783, -0.6903,  0.6304, -0.9963],
        [ 0.5715, -0.8006,  0.2073, -0.9556, -0.6067]])

In [7]:
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5%非零项
dgl.from_scipy(spmat) 

import networkx as nx
nx_g = nx.path_graph(5) # 一条链路0-1-2-3-4
dgl.from_networkx(nx_g) # 来自NetworkX

nxg = nx.DiGraph([(2, 1), (1, 2), (2, 3), (0, 0)])
dgl.from_networkx(nxg)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})

In [25]:
# 创建一个具有3种节点类型和3种边类型的异构图
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)
g.metagraph().edges()

g.num_nodes()
g.num_nodes('drug')

g.nodes['drug'].data['hv'] = th.ones(3, 1)
g.nodes['drug'].data['hv']

g.edges['treats'].data['he'] = th.zeros(1, 1)
g.edges['treats'].data['he']



10

In [29]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'is similar', 'drug'): (th.tensor([0, 1]), th.tensor([2, 3]))
})
g.ndata['hv'] = th.ones(4, 1)

g.ndata['hv']

tensor([[1.],
        [1.],
        [1.],
        [1.]])

In [40]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
})

g.nodes['drug'].data['hv'] = th.ones(3, 1)
g.nodes['disease'].data['hv'] = th.ones(3, 1)
g.nodes['disease'].data['hu'] = th.ones(3, 1)

eg = dgl.edge_type_subgraph(g, [('drug', 'interacts', 'drug'),
                                ('drug', 'treats', 'disease')])

eg.nodes['drug'].data['hv']

g.ntypes



['disease', 'drug', 'gene']

In [59]:
g = dgl.heterograph({
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))})
g.nodes['drug'].data['hv'] = th.zeros(4, 1)
g.nodes['disease'].data['hv'] = th.ones(3, 1)
g.edges['interacts'].data['he'] = th.zeros(2, 1)
g.edges['treats'].data['he'] = th.zeros(1, 2)

hg = dgl.to_homogeneous(g, ndata=['hv'])

hg.ndata[dgl.NTYPE]

hg.ndata[dgl.NID]


device(type='cpu')

In [60]:

g.device

device(type='cpu')